In [1]:
import pandas as pd
import numpy as np 
import os

In [4]:
path = r'D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn'
for dirname, _,filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(path,filename))

D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn\datasets_63741_124191_links.csv
D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn\datasets_63741_124191_movies.csv
D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn\datasets_63741_124191_tags.csv
D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn\ratings.csv
D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn\Untitled.ipynb
D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn\Untitled-checkpoint.ipynb


In [5]:
movie_path = r'D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn\datasets_63741_124191_movies.csv'
rating_path = r'D:\C++\PYTHON\PYTHON_PRACTICE\recommendation_system\collaborative-filtering\knn\ratings.csv'

In [7]:
movies = pd.read_csv(movie_path,
                    usecols=['movieId','title'],
                    dtype={'movieId': 'int32', 'title': 'str'})
rating = pd.read_csv(rating_path,
                    usecols=['userId', 'movieId', 'rating'],
                    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [8]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [9]:
rating.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
# Users have rated many different movies

In [12]:
# MErgering the rating and movies df
df = pd.merge(rating,movies,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [13]:
df.shape

(100836, 4)

In [14]:
df.isna().sum()

userId     0
movieId    0
rating     0
title      0
dtype: int64

In [15]:
combine_movie_rating = df.dropna(axis=0, subset=['title'])

In [16]:
combine_movie_rating.shape

(100836, 4)

In [96]:
# Counting the ratings for each movie
movie_rating_count = (combine_movie_rating.groupby(by=['title']))

In [97]:
movie_rating_count.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997)
100832,610,160527,4.5,Sympathy for the Underdog (1971)
100833,610,160836,3.0,Hazard (2005)
100834,610,163937,3.5,Blair Witch (2016)


In [98]:
movie_rating_count= movie_rating_count['rating'].count().reset_index().rename(columns=
                                                         {'rating':'totalRatingCount'})

In [99]:
movie_rating_count.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [34]:
# Finding the total rating count for each movie

rating_with_total_rating_count = combine_movie_rating.merge(
movie_rating_count,left_on='title',right_on='title',how='left')
rating_with_total_rating_count.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [35]:
movie_rating_count['totalRatingCount'].describe()

count    9719.000000
mean       10.375141
std        22.406220
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: totalRatingCount, dtype: float64

In [36]:
# Selecting the movies with atleast 50 rating 
popularity_threshold = 50
rating_of_popular_movies = rating_with_total_rating_count[
    rating_with_total_rating_count['totalRatingCount']>=popularity_threshold
]

In [37]:
rating_of_popular_movies.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [38]:
rating_of_popular_movies.shape

(41362, 5)

In [39]:
movie_feature = rating_of_popular_movies.pivot_table(
index='title', columns='userId', values='rating').fillna(0)
movie_feature.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [40]:
from scipy.sparse import csr_matrix

movies_feature_csr_df = csr_matrix(movie_feature.values)

In [43]:
from sklearn.neighbors import NearestNeighbors

In [44]:
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(movies_feature_csr_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [46]:
movie_feature.shape

(450, 606)

In [79]:
def recommend_movies(index):
    # distance and the indeics of 6 neighbors for the given movie index
    distance, indice = model_knn.kneighbors(movie_feature.iloc[index,:].values.reshape(1,-1),n_neighbors=6)
    # this returns an array of size(1,6), so this need to be flattened 
    
    for i in range(0,len(distance.flatten())):
        if i == 0:
            print('Recommendation for {0}: \n'.format
                 (movie_feature.index[index]))
        else:
            print('{0}: {1}, with distance of {2}'.format(
            i,movie_feature.index[indice.flatten()[i]],
                 distance.flatten()[i]))

In [80]:
recommend_movies(index=123)

Recommendation for Departed, The (2006): 

1: Prestige, The (2006), with distance of 0.3903629779815674
2: Dark Knight, The (2008), with distance of 0.3990795612335205
3: Catch Me If You Can (2002), with distance of 0.3992686867713928
4: Batman Begins (2005), with distance of 0.4044152498245239
5: Fight Club (1999), with distance of 0.4118533134460449
